## Artist Prediction

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing

In [20]:
INPUT_DATA_PATH = 'data/spotify_features.csv'

INPUT_DATA_COLUMN_NAMES = [
  'Genre', 'Artist Name', 'Track Name', 'Track ID', 'Popularity', 'Acousticness',
  'Danceability', 'Duration (ms)', 'Energy', 'Instrumentalness', 'Key', 'Liveness',
  'Loudness', 'Mode', 'Speechiness', 'Tempo', 'Time Signature', 'Valence'
]

In [21]:
INPUT_DATA_COLUMNS_TO_USE = [
  'Acousticness',
  'Danceability',
  'Energy',
  'Instrumentalness',
  'Liveness',
  'Loudness',
  'Speechiness',
  'Tempo',
  'Valence',
  # categorical columns
  'Artist Name',
  'Key',
  'Mode',
  'Time Signature'
]

In [22]:
NUMERIC_COLUMNS_TO_SCALE = ['Loudness', 'Tempo']
CATEGORICAL_COLUMNS_TO_ONE_HOT_ENCODE = ['Key', 'Mode', 'Time Signature']
IDENTIFYING_COLUMNS = ['Artist Name', 'Track Name']

In [23]:
def load_input_data(include_identifying_columns = False):
  columns_to_use = (IDENTIFYING_COLUMNS + INPUT_DATA_COLUMNS_TO_USE) if include_identifying_columns else INPUT_DATA_COLUMNS_TO_USE

  data_frame = pd.read_csv(
    INPUT_DATA_PATH,
    header = 0,
    names = INPUT_DATA_COLUMN_NAMES,
    usecols = columns_to_use
  )
  data_frame = data_frame[columns_to_use]

  return data_frame

In [24]:
def preprocess_input_data(data_frame):
  z_score_scaler = sklearn.preprocessing.StandardScaler(copy = True)
  data_frame[NUMERIC_COLUMNS_TO_SCALE] = z_score_scaler.fit_transform(data_frame[NUMERIC_COLUMNS_TO_SCALE].to_numpy())

  data_frame = pd.get_dummies(
    data_frame,
    columns = CATEGORICAL_COLUMNS_TO_ONE_HOT_ENCODE,
    sparse = False
  )

  return data_frame


In [25]:
dfx = load_input_data()
dfx = preprocess_input_data(dfx)
dfx.head()

,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence,Artist Name,...,Key_F#,Key_G,Key_G#,Mode_Major,Mode_Minor,Time Signature_0/4,Time Signature_1/4,Time Signature_3/4,Time Signature_4/4,Time Signature_5/4
0,0.611,0.389,0.910,0.000,0.3460,1.290703,0.0525,1.595607,0.814,Henri Salvador,...,0,0,0,1,0,0,0,0,1,0
1,0.246,0.590,0.737,0.000,0.1510,0.668683,0.0868,1.823253,0.816,Martin & les fées,...,1,0,0,0,1,0,0,0,1,0
2,0.952,0.663,0.131,0.000,0.1030,-0.718402,0.0362,-0.588326,0.368,Joseph Williams,...,0,0,0,0,1,0,0,0,0,1
3,0.703,0.240,0.326,0.000,0.0985,-0.434817,0.0395,1.750597,0.227,Henri Salvador,...,0,0,0,1,0,0,0,0,1,0
4,0.950,0.331,0.225,0.123,0.2020,-1.930601,0.0456,0.741433,0.390,Fabien Nataf,...,0,0,0,1,0,0,0,0,1,0


In [26]:
unique = dfx['Artist Name'].unique()

In [27]:
len(unique)

14564

In [28]:
import torch
from keras.utils import np_utils
torch.autograd.set_detect_anomaly(True)
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import (
    Dataset,
    DataLoader,
)


label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(unique)

class CustomDataset(Dataset):
    def __init__ (self, dfx):
        df = dfx.copy()
        artist = df['Artist Name']
        del df['Artist Name']
        X = df.values
        artist = label_encoder.fit_transform(artist)
        self.targets = np_utils.to_categorical(artist)
        self.data = X.astype(np.float32)
    
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, index):
        return torch.tensor(self.data[index]), torch.tensor(self.targets[index])

ds = CustomDataset(dfx)

In [29]:
ds.data.shape, ds.targets.shape

((232725, 28), (232725, 14564))

In [30]:
from sklearn import model_selection
train_data_set, test_data_set = model_selection.train_test_split(
ds,
test_size = 1000,
shuffle = True
)

train_data_loader = DataLoader(
train_data_set,
batch_size = 128,
shuffle = True
)

test_data_loader = DataLoader(
test_data_set,
batch_size = 32,
shuffle = True
)

train_size = len(train_data_loader.dataset)
val_size = len(test_data_loader.dataset)

In [31]:
import torch

class ClassificationModel(torch.nn.Module):
  def __init__(self):
    super(ClassificationModel, self).__init__()

    self.linear1 = torch.nn.Linear(ds[0][0].shape[0], 512)
    self.relu1 = torch.nn.ReLU()
    self.linear2 = torch.nn.Linear(512, len(unique))

  def forward(self, data):
    data = data.float()

    data = self.linear1(data)
    data = self.relu1(data)
    data = self.linear2(data)

    return data

In [32]:
import torch
import numpy as np

class EarlyStopping:
    def __init__(self, patience=7, verbose=True, delta=0, path='model.pth', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model, 'artist_model.pth')
        self.val_loss_min = val_loss

In [33]:
import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ClassificationModel()
model.to(device)

criterion = torch.nn.CrossEntropyLoss()


def fit(num_epochs, learning_rate, model, train_loader, val_loader):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    train_losses = []
    valid_losses = []
    avg_train_losses = []
    avg_valid_losses = [] 

    early_stopping = EarlyStopping(patience=10, verbose=True)

    for epoch in range(num_epochs):
        losses = []
        correct = 0
        
        model.train()
        for (data, targets) in tqdm.tqdm(train_loader, desc= "Training"):
            data = data.to(device)
            targets = targets.to(device)
            
            optimizer.zero_grad()
            
            scores = model(data)
            loss = criterion(scores, targets)
            losses.append(loss.item())
            
            loss.backward()

            torch.nn.utils.clip_grad_value_(model.parameters(), 0.1)

            optimizer.step()

            correct += (scores.argmax(dim=1) == targets.argmax(dim=1)).float().sum()
            train_losses.append(loss.item())

        optimizer.zero_grad()
        
        model.eval()
        correct_ = 0
        for data, target in tqdm.tqdm(val_loader, desc= "Evaluation"):
          data = data.to(device)
          target = target.to(device)
          output = model(data)            
          loss = criterion(output, target)
          valid_losses.append(loss.item())
          correct_ += (output.argmax(dim=1) == target.argmax(dim=1)).float().sum()
        
        accuracy = 100 * correct / train_size
        val_accuracy = 100 * correct_ / val_size

        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        
        
        print(f"{epoch+1} || valid_loss: {valid_loss} | train_loss: {train_loss} | accuracy: {accuracy} | val_accuracy: {val_accuracy}")
        
        early_stopping(valid_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
            
        # clear lists to track next epoch
        train_losses = []
        valid_losses = []

def check_accuracy(loader, model):
    print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in tqdm.tqdm(loader):
            x = x.to(device)
            y = y.to(device)

            scores = model(x)
            num_correct += (scores.argmax(dim=1) == y.argmax(dim=1)).sum()
            num_samples += 1*x.shape[0]

        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")

    model.train()

In [34]:
lr = 0.001
epochs = 1000
history1 = fit(epochs, lr, model, train_data_loader, test_data_loader)

Evaluation: 100%|██████████| 32/32 [00:00<00:00, 74.39it/s]


1 || valid_loss: 7.021359324455261 | train_loss: 7.598760606174901 | accuracy: 2.617326498031616 | val_accuracy: 4.400000095367432
Validation loss decreased (inf --> 7.021359).  Saving model ...


Evaluation: 100%|██████████| 32/32 [00:00<00:00, 77.72it/s]


2 || valid_loss: 6.657011613249779 | train_loss: 6.3480575965426365 | accuracy: 5.026648044586182 | val_accuracy: 6.300000190734863
Validation loss decreased (7.021359 --> 6.657012).  Saving model ...


Evaluation: 100%|██████████| 32/32 [00:00<00:00, 75.31it/s]


...

11 || valid_loss: 7.504869386553764 | train_loss: 3.6879726467311613 | accuracy: 26.417951583862305 | val_accuracy: 14.899999618530273
EarlyStopping counter: 8 out of 10


Evaluation: 100%|██████████| 32/32 [00:00<00:00, 75.14it/s]


12 || valid_loss: 7.826947197318077 | train_loss: 3.5622746656113473 | accuracy: 28.182544708251953 | val_accuracy: 15.600000381469727
EarlyStopping counter: 9 out of 10


Evaluation: 100%|██████████| 32/32 [00:00<00:00, 68.37it/s]

13 || valid_loss: 7.817012548446655 | train_loss: 3.449610567961787 | accuracy: 29.900527954101562 | val_accuracy: 16.700000762939453
EarlyStopping counter: 10 out of 10
Early stopping


In [36]:
check_accuracy(test_data_loader, torch.load('artist_model.pth'))

Checking accuracy on test data


100%|██████████| 32/32 [00:00<00:00, 109.10it/s]

Got 75 / 1000 with accuracy 7.50


## Genre Prediction

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing

In [2]:
INPUT_DATA_PATH = 'data/spotify_features.csv'

INPUT_DATA_COLUMN_NAMES = [
  'Genre', 'Artist Name', 'Track Name', 'Track ID', 'Popularity', 'Acousticness',
  'Danceability', 'Duration (ms)', 'Energy', 'Instrumentalness', 'Key', 'Liveness',
  'Loudness', 'Mode', 'Speechiness', 'Tempo', 'Time Signature', 'Valence'
]

In [3]:
INPUT_DATA_COLUMNS_TO_USE = [
  'Acousticness',
  'Danceability',
  'Energy',
  'Instrumentalness',
  'Liveness',
  'Loudness',
  'Speechiness',
  'Tempo',
  'Valence',
  # Categorical features
  'Genre',
  'Key',
  'Mode',
  'Time Signature'
]

In [4]:
NUMERIC_COLUMNS_TO_SCALE = ['Loudness', 'Tempo']
CATEGORICAL_COLUMNS_TO_ONE_HOT_ENCODE = ['Key', 'Mode', 'Time Signature']
IDENTIFYING_COLUMNS = ['Artist Name', 'Track Name']

In [5]:
def load_input_data(include_identifying_columns = False):
  columns_to_use = (IDENTIFYING_COLUMNS + INPUT_DATA_COLUMNS_TO_USE) if include_identifying_columns else INPUT_DATA_COLUMNS_TO_USE

  data_frame = pd.read_csv(
    INPUT_DATA_PATH,
    header = 0,
    names = INPUT_DATA_COLUMN_NAMES,
    usecols = columns_to_use
  )
  final_genres = ['R&B', 'Alternative', 'Country', 'Dance','Electronic', 'Folk', 'Blues', 'Opera', 'Hip-Hop','Rap', 'Indie','Classical', 'Pop', 'Reggae', 'Jazz', 'Rock']
  data_frame = data_frame[data_frame['Genre'].isin(final_genres)]
  data_frame = data_frame[columns_to_use]

  return data_frame

In [6]:
def preprocess_input_data(data_frame):
  z_score_scaler = sklearn.preprocessing.StandardScaler(copy = True)
  data_frame[NUMERIC_COLUMNS_TO_SCALE] = z_score_scaler.fit_transform(data_frame[NUMERIC_COLUMNS_TO_SCALE].to_numpy())
  data_frame = pd.get_dummies(
    data_frame,
    columns = CATEGORICAL_COLUMNS_TO_ONE_HOT_ENCODE,
    sparse = False
  )

  return data_frame


In [7]:
data_framex = load_input_data()
data_framex = preprocess_input_data(data_framex)

In [8]:
data_framex.head()

,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence,Genre,...,Key_F#,Key_G,Key_G#,Mode_Major,Mode_Minor,Time Signature_0/4,Time Signature_1/4,Time Signature_3/4,Time Signature_4/4,Time Signature_5/4
135,0.0830,0.724,0.689,0.000000,0.3040,0.567378,0.1350,0.943281,0.6930,R&B,...,0,0,0,0,1,0,0,0,1,0
136,0.3230,0.685,0.610,0.000000,0.1020,0.687692,0.0439,-0.793991,0.3230,R&B,...,0,0,0,0,1,0,0,1,0,0
137,0.0675,0.762,0.520,0.000004,0.1140,0.684946,0.0959,-1.438634,0.0862,R&B,...,0,0,0,0,1,0,0,0,1,0
138,0.3600,0.563,0.366,0.002430,0.0955,0.400208,0.1210,-1.095093,0.7680,R&B,...,0,0,0,0,1,0,0,0,1,0
139,0.5960,0.653,0.621,0.000000,0.0811,0.601876,0.0409,-0.606569,0.4660,R&B,...,0,0,0,0,1,0,0,0,1,0


In [9]:
unique = data_framex['Genre'].unique()

In [10]:
from sklearn.preprocessing import LabelEncoder
import torch
from keras.utils import np_utils
torch.autograd.set_detect_anomaly(True)

from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import (
    Dataset,
    DataLoader,
)
label_encoder = LabelEncoder()


class CustomDataset(Dataset):
    def __init__ (self, df):
        genre = df['Genre']
        del df['Genre']
        X = df.values
        Y = genre

        self.data = X.astype(np.float32)
        
        self.target = label_binarize(Y, classes = unique).astype(np.float32)
    
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, index):
        return torch.tensor(self.data[index]), torch.tensor(self.target[index])

ds = CustomDataset(data_framex)

In [11]:
from sklearn import model_selection
train_data_set, test_data_set = model_selection.train_test_split(
ds,
test_size = 1000,
shuffle = True
)

train_data_loader = DataLoader(
train_data_set,
batch_size = 500,
shuffle = True
)

test_data_loader = DataLoader(
test_data_set,
batch_size = 500,
shuffle = True
)

In [12]:
train_size = len(train_data_loader.dataset)
val_size = len(test_data_loader.dataset)

In [13]:
import torch

class ClassificationModel(torch.nn.Module):
  def __init__(self):
    super(ClassificationModel, self).__init__()

    self.linear1 = torch.nn.Linear(ds[0][0].shape[0], 512)
    self.relu1 = torch.nn.ReLU()
    self.linear2 = torch.nn.Linear(512, len(unique))

  def forward(self, data):
    data = data.float()

    data = self.linear1(data)
    data = self.relu1(data)
    data = self.linear2(data)

    return data

In [14]:
import torch
import numpy as np

class EarlyStopping:
    def __init__(self, patience=7, verbose=True, delta=0, path='model.pth', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model, 'model.pth')
        self.val_loss_min = val_loss

In [15]:
import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ClassificationModel()
model.to(device)

criterion = torch.nn.CrossEntropyLoss()


def fit(num_epochs, learning_rate, model, train_loader, val_loader):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    train_losses = []
    valid_losses = []
    avg_train_losses = []
    avg_valid_losses = [] 

    early_stopping = EarlyStopping(patience=10, verbose=True)

    for epoch in range(num_epochs):
        losses = []
        correct = 0
        
        model.train()
        for (data, targets) in tqdm.tqdm(train_loader, desc= "Training"):
            data = data.to(device)
            targets = targets.to(device)
            
            optimizer.zero_grad()
            
            scores = model(data)
            loss = criterion(scores, targets)
            losses.append(loss.item())
            
            loss.backward()

            torch.nn.utils.clip_grad_value_(model.parameters(), 0.1)

            optimizer.step()

            correct += (scores.argmax(dim=1) == targets.argmax(dim=1)).float().sum()
            train_losses.append(loss.item())

        optimizer.zero_grad()

        model.eval()
        correct_ = 0
        for data, target in tqdm.tqdm(val_loader, desc= "Evaluation"):
          data = data.to(device)
          target = target.to(device)
          output = model(data)            
          loss = criterion(output, target)
          valid_losses.append(loss.item())
          correct_ += (output.argmax(dim=1) == target.argmax(dim=1)).float().sum()
        
        accuracy = 100 * correct / train_size
        val_accuracy = 100 * correct_ / val_size

        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        
        
        print(f"{epoch+1} || valid_loss: {valid_loss} | train_loss: {train_loss} | accuracy: {accuracy} | val_accuracy: {val_accuracy}")
        
        early_stopping(valid_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
            
        # clear lists to track next epoch
        train_losses = []
        valid_losses = []

def check_accuracy(loader, model):
    print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in tqdm.tqdm(loader):
            x = x.to(device)
            y = y.to(device)

            scores = model(x)
            num_correct += (scores.argmax(dim=1) == y.argmax(dim=1)).sum()
            num_samples += 1*x.shape[0]

        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")

    model.train()

In [16]:
lr = 0.001
epochs = 1000
history1 = fit(epochs, lr, model, train_data_loader, test_data_loader)

Evaluation: 100%|██████████| 2/2 [00:00<00:00, 180.60it/s]


1 || valid_loss: 2.085245966911316 | train_loss: 2.2270821760440693 | accuracy: 26.011947631835938 | val_accuracy: 30.700000762939453
Validation loss decreased (inf --> 2.085246).  Saving model ...


Evaluation: 100%|██████████| 2/2 [00:00<00:00, 118.81it/s]


2 || valid_loss: 2.0523831844329834 | train_loss: 2.049395271827435 | accuracy: 30.686141967773438 | val_accuracy: 30.700000762939453
Validation loss decreased (2.085246 --> 2.052383).  Saving model ...


Evaluation: 100%|██████████| 2/2 [00:00<00:00, 211.46it/s]


...

65 || valid_loss: 1.918606162071228 | train_loss: 1.8524111036596627 | accuracy: 36.31547927856445 | val_accuracy: 34.099998474121094
EarlyStopping counter: 8 out of 10


Evaluation: 100%|██████████| 2/2 [00:00<00:00, 124.25it/s]


66 || valid_loss: 1.9210617542266846 | train_loss: 1.8510285155526522 | accuracy: 36.3665885925293 | val_accuracy: 33.79999923706055
EarlyStopping counter: 9 out of 10


Evaluation: 100%|██████████| 2/2 [00:00<00:00, 114.26it/s]

67 || valid_loss: 1.9239519834518433 | train_loss: 1.8513438533092368 | accuracy: 36.341033935546875 | val_accuracy: 33.20000076293945
EarlyStopping counter: 10 out of 10
Early stopping


In [18]:
check_accuracy(test_data_loader, torch.load('model.pth'))

Checking accuracy on test data


100%|██████████| 2/2 [00:00<00:00, 44.41it/s]

Got 329 / 1000 with accuracy 32.90
